In [2]:
import pandas as pd
import cleaningtools as ct 
import file_config as fconfig
import xrf_config as xconfig
import numpy as np
import math
from tqdm.notebook import tqdm_notebook as tqdm
tqdm.pandas()
from tqdm import tqdm
import os
import file_config as fconfig
import pandas as pd
import glob
import openpyxl as pxl
import xlwings as xl
import scraper as scraper
import re

import string


check for pandas
pandas is properly installed
check for tqdm
tqdm is properly installed
check for numpy
numpy is properly installed
check for openpyxl
openpyxl is properly installed
check for xlwings


In [23]:
mac_path='Shift Reports'
fend='/**/*.xlsx'
win_path='/Users/sebastiancoombs/Library/CloudStorage/GoogleDrive-sebastianbcoombs@gmail.com/Shared drives/AMC Projects/_AZ_Kay/_Drilling/Boart Forms/Shift Reports/**/*.xlsx'
output_path='/Users/sebastiancoombs/Library/CloudStorage/GoogleDrive-sebastianbcoombs@gmail.com/Shared drives/AMC Projects/_AZ_Kay/_Drilling/Boart Forms/Shift Reports/Scraped reports/'
# scrape_path=mac_path
scrape_path=win_path

In [24]:
os.path.join(mac_path,fend)

'/**/*.xlsx'

In [25]:
files=glob.glob(scrape_path, recursive = True)
print(len(files))


785


In [26]:
file=files[32]
fname=file.split(r'/')[-1]
fname

'BK-77 9-19-22 .xlsx'

In [27]:

def integer_columns(data):
    data.columns=range(len(data.columns))
    return data



In [28]:
def find_info_index(data,name):
    # this function is used to find the location of a piece of data that you want
    col=data[data==name].dropna(axis=1,how='all').dropna(how='all').columns.values[0]
    row=data[data==name].dropna(axis=1,how='all').dropna(how='all').index.values[0]
    return row,col


In [29]:
def pull_data(data,name,n_rows=1,n_cols=1,all_collumns=False,include_header=True,one_liner=False):
    # this function is used to grab all the splices need for the get block function
    row,col=find_info_index(data,name=name)
    if not include_header:
        row =row+1
    if not all_collumns:
        slice=data.iloc[row:row+(n_rows+1),col:col+n_cols]
    else:
        slice=data.iloc[row:row+(n_rows+1),:]
    if one_liner:
        slice=data.iloc[[row],col:]
    return slice.copy().reset_index(drop=True)
    


In [30]:
def get_singles(data,name,include_header=True,one_liner=False):
    ## this grabs specific lines of the original excel file
    row,col=find_info_index(data,name=name)
    singular_data=[s for  s in data.iloc[[row],col:].dropna()]
    singles=[]
    if one_liner:
        data=pull_data(data=data,name=name,include_header=include_header,one_liner=one_liner)
        data=data.T.reset_index(drop=True).T
        data=data.dropna(axis=1)
        data=data.T.reset_index(drop=True).T

        odds=[c for c in data.columns if c%2==0]
        evens=[c for c in data.columns if c%2!=0]
        odds=data[odds].T.reset_index(drop=True).T
        evens=data[evens].T.reset_index(drop=True).T
        singles=pd.concat([odds,evens],ignore_index=True,axis=0)
        singles=singles.dropna(how='all').reset_index(drop=True)

    else:
        for s in singular_data:
            slice=pull_data(data=data,name=s,include_header=include_header,one_liner=one_liner)
            singles.append(slice)
        singles=pd.concat(singles,axis=1)
        singles=singles.reset_index(drop=True)
        singles=singles.T.set_index(0,drop=True).T

    return singles


In [31]:
def get_blocks(data,name,n_rows=9,n_cols=10,transpose=False,all_collumns=False,include_header=True):
    ## this grabs chunks of the original excel file
    data=pull_data(data,name=name,n_rows=n_rows,n_cols=n_cols,all_collumns=all_collumns,include_header=include_header)
    data=data.T.dropna(how='all',axis=0).reset_index(drop=True).T

    if transpose:
        data=data.T.reset_index(drop=True)


    return data
   

In [32]:
def get_info(data):
    ##function is only for verion1 data
    report_info=get_blocks(data,name='Report Information',n_rows=2,n_cols=5,transpose=True)
    client_info=get_blocks(data,name='Client Information',n_rows=7,n_cols=5,transpose=True)
    info=pd.concat([report_info,client_info],axis=1)
    return info

In [33]:
def write_excel(data_dict,file):
    ## this writes new sheets to an existng file
    fname=file.split(r'/')[-1]
    writer = pd.ExcelWriter(file,mode='a',  engine = 'openpyxl',if_sheet_exists='replace')

    for name in data_dict:
        try:
            # print(f'writing {name} to {fname}')
            data_dict[name].to_excel(writer, sheet_name = name,index=False)
        except Exception as e:
            print(e)
            print(f'Error writing {name} to {fname}')
            print(e)
    writer.close()
    return


In [34]:
def fix_drilling_header(data):
    ## this is only used on the version 2 data
    ## it merges the gross format of the drilling info in the version 2 headers into something useable
    header=get_blocks(data=data,name='Footage Summary',all_collumns=True,n_rows=2,include_header=True)
    header=header.T
    header=header.fillna(method='ffill')
    header=header.fillna(' ')

    cols=[c.strip() for c in header.agg(' '.join,axis=1)]
    return cols


In [35]:
def set_columns(data):
    # this sets the colums headers to the first row
    data=data.T.set_index(0,drop=True).T
    data=data.reset_index(drop=True)
    data=data.dropna(how='all',axis=0)
    return data

In [36]:
def clean_column_names(data):
    # this takes all the gross column typos and standardizes them

    try:
        data.columns=[str(c).lower() for c in data.columns]
        data.columns=[c.replace('  ',' ') for c in data.columns]
        data.columns=[c.replace('/','') for c in data.columns]
        # data.columns=[c.replace('__','_') for c in data.columns]
        data.columns=[c.replace('footage summary','') for c in data.columns]

        data.columns=[c.replace('#','') for c in data.columns]
        data.columns=[re.sub('\r?\n', '/', c )for c in data.columns]
        data.columns=[re.sub(' +/', '/', c )for c in data.columns]

        #data.columns=[c.replace('   /','/') for c in data.columns]

        data.columns=[c.replace('of','') for c in data.columns]
        data.columns=[c.replace(':','') for c in data.columns]
        data.columns=[c.replace('hole(s)','hole no.') for c in data.columns]
        data.columns=[c.strip() for c in data.columns]
        data.columns=[str(c).title() for c in data.columns]

        # data=data.groupby(level=0,axis=1).sum(numeric_only=False)

    except Exception as e:
        
        print('error in cleaning the column names')
        print(e)
    return data

In [37]:
def merge_dicts(data_dict):
    # this takes all the scraped info and merges it into its own data set
    scraped=pd.concat([data_dict[d].reset_index(drop=True) for d in data_dict],ignore_index=True)
    scraped=scraped.reset_index(drop=True)
    scraped=clean_column_names(scraped)
    return scraped

In [38]:
def fill_columns(data):
    ## fill the date and hole id columns
    fill_cols=['date','contract','location','hole no.']
    fill_cols=[f.title() for f in fill_cols]
    data[fill_cols]=data[fill_cols].fillna(method='ffill').fillna(method='bfill')
    return data

In [39]:
def cell_block(cells):
    alphabet=list(string.ascii_lowercase)
    cells=cells.lower()
    cells.split(':')
    cells.split(':')
    start=cells.split(':')[0]
    end=cells.split(':')[-1]
    start=[c for c in start]
    end=[c for c in end]
    c_start=start[0]
    c_end=end[0]
    r_start=int(''.join(start[1:]))
    r_end=int(''.join(end[1:]))

    s=alphabet.index(c_start)
    e=alphabet.index(c_end)
    n_cols=abs(e-s)+1
    n_rows=abs(r_end-r_start)+1
    return n_cols,n_rows

In [40]:
## read mapping
mapping_file='/Users/sebastiancoombs/Library/CloudStorage/GoogleDrive-sebastianbcoombs@gmail.com/Shared drives/AMC Projects/_AZ_Kay/_Drilling/Boart Forms/Shift Reports/Shift Report Version Mapping.xlsx'
map_file=pd.read_excel(mapping_file,skiprows=1)
map_file.head(1)

,Metric,Label Format,Transposed,Label Cell,"Data Format, ex.",Data Cell,Label Format.1,Transposed.1,Label Cell.1,"Data Format, ex. .1",Data Cell.1
0,Date,Date,True,N3,Aug-30-2022,O3:S3,Date,1.0,A4,2022-10-01 00:00:00,B4:D4


In [41]:

metric= map_file['Metric']
label_cols= map_file.filter(like='Label Format')
labels=map_file['Label Format'].fillna('nan')
labels2=map_file['Label Format.1'].fillna('nan')
## make mappin dicts
col_map1=dict([(m.strip(),l.strip()) for m,l in zip(labels,metric)])
col_map2=dict([(m.strip(),l.strip()) for m,l in zip(labels2,metric)])
# merge the maps
col_map=col_map1|col_map2
col_map


{'Date': 'Date',
 'Drill': 'Drill ID',
 'Hole No.': 'Hole ID',
 'Shift': 'Drill Shift',
 'Type': 'Activity Type',
 'Size': 'Bit Size',
 'Drill From': 'Drilling From_ft',
 'Drill To': 'Drilling To_ft',
 'Total Distance': 'Total Drilled_ft',
 'Volume': 'Fluid Volume_gal',
 '-': 'Fluid Volume_gal',
 'Description': 'Billable Item',
 'Unit': 'Billable Unit',
 'Quantity': 'Billable Quantity',
 'Hours': 'Activity Hours',
 'Activity': 'Activity Item',
 'Total Man Hours': 'Total Man Hours',
 'Total Activity Hours': 'Total Activity Hours',
 'Distance Drilled': 'Distance Drilled_ft',
 'Distance Drilled/Activity Hour': 'Footage Per Activity Hour',
 'Distance Drilled/Drilling Hour': 'Footge Per Drilling Hour',
 'DO NOT EDIT NAMES IN THIS COLUMN': 'IF YOU WANT TO CHANGE',
 'IT WILL BREAK THE SCAPER': 'FINAL COLUMN NAMES',
 'FEEL FREE TO ADD NAMES': 'USE THIS COLUMN',
 'nan': 'Footge Per Drilling Hour',
 'Bit Size & Type': 'Bit Size',
 'Drilling From': 'Drilling From_ft',
 'Drilling To': 'Drilling To

In [42]:
def scrape_V2(df):
    data_dict={}
    try:
        # singulars
        name='DATE:'
        data=get_singles(data=df,name=name,one_liner=True)
        data=set_columns(data)
        name='Info'
        data_dict[name]=data
        
    except Exception as e:
        print(e)
        print('FAILED ON ',name)

    try:
        name='# of Loads'
        data=get_blocks(data=df,name=name,n_rows=0,n_cols=5,transpose=True)
        data=set_columns(data)
        data['Volume']=data['# of Loads']*1500
        data_dict[name]=data


    except Exception as e:
        print(e)
        print('FAILED ON ',name)

    try:
        ### special stuff for footages
        name='Footage Summary'
        cols=fix_drilling_header(data=df)
        data=get_blocks(data=df,name=name,all_collumns=True,n_rows=7,include_header=False)
        data=data.drop([0,1])
        data.columns=cols
        data_dict[name]=data

    except Exception as e:
        print(e)
        print('FAILED ON ',name)

    # try:
    #     ## hourly distribution
    #     name='Hourly Distribution'
    #     data=get_blocks(data=df,name=name,n_rows=27,n_cols=6,all_collumns=False,include_header=True,transpose=False)
    #     data=set_columns(data)
    #     data_dict[name]=data

    # except Exception as e:
    #     print(e)
    #     print('FAILED ON ',name)

    # try:
    #     ## drill bits
    #     name='Bits'
    #     data=get_blocks(data=df,name=name,n_rows=6,n_cols=10,all_collumns=False,include_header=True,transpose=False)
    #     data=set_columns(data)
    #     data_dict[name]=data

    # except Exception as e:
    #     print(e)
    #     print('FAILED ON ',name)

    # try:
    #     ## Equipment
    #     name='Equip. Description'
    #     data=get_blocks(data=df,name=name,n_rows=10,n_cols=10,all_collumns=False,include_header=True,transpose=False)
    #     data=set_columns(data)
    #     data_dict[name]=data

    # except Exception as e:
    #     print(e)
    #     print('FAILED ON ',name)

    # try:
    #     ## Remarks and dillers get added together
    #     name='Remarks: (Note any problems from pre-shift inspection here)'
    #     data2=get_blocks(data=df,name=name,n_rows=3,all_collumns=True,include_header=True,transpose=False)
    #     data2=set_columns(data2)

    #     name='Drillers'
    #     data=get_blocks(data=df,name=name,n_rows=3,all_collumns=True,include_header=True,transpose=False)
    #     data=set_columns(data)
    #     data=pd.concat([data,data2],axis=1)
    #     data_dict[name]=data

    # except Exception as e:
    #     print(e)
    #     print('FAILED ON ',name)

    # try:
    #     ## This is the Billables
    #     name='Description'
    #     data=get_blocks(data=df,name=name,n_rows=10,n_cols=10,include_header=True,transpose=False)
    #     data=set_columns(data)
    #     name='Products'
    #     data_dict[name]=data

    # except Exception as e:
    #     print(e)
    #     print('FAILED ON ',name)
    return data_dict



In [43]:
def scrape_V1(df):
    data_dict={}
    
    try:
        ## Water
        name='Fluids'
        data=get_blocks(df,name=name,n_rows=9,n_cols=4,include_header=False)
        data=set_columns(data)
        name='Water'
        data_dict[name]=data
    except Exception as e:
        print(e)
        print('FAILED ON ',name)
    try:
        ## Billable
        name='Billable Quantities'
        data=get_blocks(df,name=name,n_rows=24,n_cols=6,include_header=False)
        data=set_columns(data)
        name='Billables'
        data_dict[name]=data
    except Exception as e:
        print(e)
        print('FAILED ON ',name)
    try:
        ## Activities
        name='Activities'
        data=get_blocks(df,name=name,n_rows=23,n_cols=10,include_header=False)
        data=set_columns(data)
        data_dict[name]=data
    except Exception as e:
        print(e)
        print('FAILED ON ',name)
    try:
        ## Info
        name='Report Information'
        data=get_blocks(df,name=name,n_rows=2,n_cols=5,transpose=True,include_header=False)
        data=set_columns(data)
        data_dict[name]=data

        name='Client Information'
        data2=get_blocks(df,name=name,n_rows=5,n_cols=4,transpose=True,include_header=False)
        data2=set_columns(data2)
        name='Info'
        data=pd.concat([data,data2],axis=1).dropna(axis=1)
        data_dict[name]=data

    except Exception as e:
        print(e)
        print('FAILED ON ',name)
    try:
        ## Hours
        name='Total Man Hours'
        data=get_blocks(df,name=name,n_rows=1,all_collumns=True,include_header=True)
        data=set_columns(data)
        data_dict[name]=data

        #personel
        name='Name'
        data=get_blocks(df,name=name,n_rows=7,all_collumns=True,include_header=True)
        data=data.dropna(axis=1)
        data=set_columns(data)
        name='Personel'
        data_dict[name]=data
    except Exception as e:
        print(e)
        print('FAILED ON',name)
    return data_dict

    

In [44]:
#Version 2 params
# load the data
file=files[32]
wb=xl.Book(file)
data_sheets=pd.ExcelFile(file)
sheet_names=data_sheets.sheet_names
data=pd.read_excel(file,sheet_name=sheet_names[0])
df=integer_columns(data=data)
data_dict={}
df.shape

(55, 14)

In [45]:
data_dict=scrape_V2(df)


In [46]:
name='# of Loads'
data=get_blocks(data=df,name=name,n_rows=0,n_cols=5,transpose=True)
data=set_columns(data)
# data=pd.DataFrame(data.sum()).T
data['Volume']=data['# of Loads']*1500
data

,# of Loads,Volume
0,1,1500
1,1,1500


In [47]:
scraped=merge_dicts(data_dict)
scraped=fill_columns(scraped)
scraped=scraped.rename(col_map,axis=1)
scraped=scraped[[c for c in metric if c.strip() in scraped.columns]]
scraped

,Date,Hole ID,Drill Shift,Bit Size,Drilling From_ft,Drilling To_ft,Total Drilled_ft,Fluid Volume_gal,Water (# of Loads)
0,2022-09-19,KM-22-81C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-09-19,KM-22-81C,NaN,NaN,NaN,NaN,NaN,1500,1
2,2022-09-19,KM-22-81C,NaN,NaN,NaN,NaN,NaN,1500,1
3,2022-09-19,KM-22-81C,Day,HQ,2665,2722,57,NaN,NaN
4,2022-09-19,KM-22-81C,A,NaN,NaN,NaN,NaN,NaN,NaN
5,2022-09-19,KM-22-81C,Aft,NaN,NaN,NaN,NaN,NaN,NaN
6,2022-09-19,KM-22-81C,B,NaN,NaN,NaN,NaN,NaN,NaN
7,2022-09-19,KM-22-81C,Night,HQ,2722,2777,55,NaN,NaN
8,2022-09-19,KM-22-81C,C,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
#Version 1 params
# load the data
file=files[-3]
v1=[v for v in files if '1_1.16.BK-75' in v]
file=v1[0]

wb=xl.Book(file)
data_sheets=pd.ExcelFile(file)
sheet_names=data_sheets.sheet_names
data=pd.read_excel(file,sheet_name=sheet_names[0])
df=integer_columns(data=data)
data_dict={}
df.shape
file

'/Users/sebastiancoombs/Library/CloudStorage/GoogleDrive-sebastianbcoombs@gmail.com/Shared drives/AMC Projects/_AZ_Kay/_Drilling/Boart Forms/Shift Reports/BK-75/Kay Mine 1_1.16.BK-75.xlsx'

In [55]:
name='Total Man Hours'
data=get_blocks(df,name=name,n_rows=1,all_collumns=True,include_header=True)
data=set_columns(data)
data

,Total Man Hours,Total Activity Hours,Distance Drilled,Distance Drilled / \nActivity Hour,Distance Drilled / \nDrilling Hour
0,72,24,74 ft,3.1,14.8


In [27]:
data_dict=scrape_V1(df)
scraped=merge_dicts(data_dict)
scraped=fill_columns(scraped)
scraped=scraped.rename(col_map,axis=1)
scraped=scraped[[c.strip() for c in metric if c.strip() in scraped.columns]]
scraped

,Date,Drill ID,Hole ID,Drill Shift,Fluid Volume_gal,Billable Item,Billable Unit,Billable Quantity,Activity Hours,Activity Item,Activity Type,Total Man Hours,Total Activity Hours,Distance Drilled_ft,Footage Per Activity Hour,Footge Per Drilling Hour
0,Apr-20-2022,NaN,KM-22-74,Day,1500 gal,NaN,NaN,NaN,NaN,NaN,Water,NaN,NaN,NaN,NaN,NaN
1,Apr-20-2022,NaN,KM-22-74,Night,1500 gal,NaN,NaN,NaN,NaN,NaN,Water,NaN,NaN,NaN,NaN,NaN
2,Apr-20-2022,NaN,KM-22-74,NaN,NaN,Operating Rate,Per Activity Hour,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apr-20-2022,NaN,KM-22-74,NaN,NaN,Core HQ Diamond Core Bit 0ft - 500ft,Per Foot,123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apr-20-2022,NaN,KM-22-74,NaN,NaN,Core PQ Diamond Core Bit 0ft - 500ft,Per Foot,59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Apr-20-2022,NaN,KM-22-74,NaN,NaN,Supervisor Hours,Per Unit,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Apr-20-2022,NaN,KM-22-74,NaN,NaN,Antonio Trevizo - Driller Assistant,Per Diem,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Apr-20-2022,NaN,KM-22-74,NaN,NaN,Arlex Rodrigue Komba Kaya - Driller Assistant,Per Diem,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Apr-20-2022,NaN,KM-22-74,NaN,NaN,Daniel Dodge - Supervisor,Per Diem,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Apr-20-2022,NaN,KM-22-74,NaN,NaN,Gabe Duarte - Driller,Per Diem,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
X=[]
Y=[]
all_data=[]
metric= map_file['Metric']
label_cols= map_file.filter(like='Label Format')
labels=map_file['Label Format'].fillna('nan')
labels2=map_file['Label Format.1'].fillna('nan')
## make mapping dicts
col_map1=dict([(m.strip(),l.strip()) for m,l in zip(labels,metric)])
col_map2=dict([(m.strip(),l.strip()) for m,l in zip(labels2,metric)])
# merge the mappings
col_map=col_map1|col_map2
test_file='Daily_scraped.xlsx'
for file in files:

    fname=file.split(r'/')[-1]
    fbase=fname.split('.')[:-1]
    fbase='_'.join(fbase)
    if 'Version' in fbase:
        continue
    print('----------------------------------------')
    print(fbase)
    print('----------------------------------------')

    try:
        data_sheets=pd.ExcelFile(file,engine='openpyxl')
        sheet_names=data_sheets.sheet_names
        data=pd.read_excel(file,sheet_name=sheet_names[0])
        df=integer_columns(data=data)
        print('shape:',df.shape)
        x,y=df.shape
        if x>60:
            data_dict=scrape_V1(df=df)
        else:
            data_dict=scrape_V2(df=df)
        scraped=merge_dicts(data_dict)
        scraped=fill_columns(scraped)
        scraped=scraped.rename(col_map,axis=1)
        scraped=scraped[[c.strip() for c in metric if c.strip() in scraped.columns]]
        scraped['file']=fname
        scraped.to_csv(output_path+fbase+'.csv',index=False)
        all_data.append(scraped)

        # write_excel(data_dict=data_dict,file=file)
        X.append(x)
        Y.append(y)
    except Exception as e:
        print(e)

        print('broken file',file)


----------------------------------------
BK-77 11-12-22
----------------------------------------
shape: (55, 14)
----------------------------------------
Kay Mine 3_7_19_BK-77
----------------------------------------
shape: (65, 19)
----------------------------------------
BK-77 10-4-22
----------------------------------------
shape: (55, 14)
----------------------------------------
Kay Mine 3_5_28_BK-77
----------------------------------------
shape: (63, 18)
----------------------------------------
BK-77 9-17-22
----------------------------------------
shape: (55, 14)
----------------------------------------
Kay Mine 3_2_2_BK-77
----------------------------------------
shape: (63, 18)
----------------------------------------
Kay Mine 3_8_01_BK-77
----------------------------------------
shape: (65, 19)
----------------------------------------
Kay Mine 3_6_9_BK-77
----------------------------------------
shape: (63, 18)
----------------------------------------
Kay Mine 3_7_05_BK-77
--

In [29]:
all_df=pd.concat(all_data,axis=0)
all_df['Date']=pd.to_datetime(all_df['Date']).dt.strftime("%m/%d/%Y")

file='Test_scraped.xlsx'
data_dict={}
common_cols=['Hole ID','Date']
all_df=all_df.sort_values(common_cols)

In [30]:
all_df.to_excel('Test_scraped.xlsx')
common_cols=['Date','Hole ID']
all_df.shape

(16105, 22)

In [31]:
shifts=all_df[all_df['Drill Shift'].notna()==True].dropna(how='all',axis=1)
shifts=shifts.groupby(['Date','Hole ID','Drill Shift']).sum(numeric_only=False).reset_index(level=[0,1,2])
# shifts=shifts.replace(0,'')
shifts

,Date,Hole ID,Drill Shift,Bit Size,Drilling From_ft,Drilling To_ft,Total Drilled_ft,Fluid Volume_gal,file,Billable Item,Billable Unit,Billable Quantity,Activity Hours,Activity Item,Activity Type
0,01/14/2022,KM-21-59,Day,0,0,0,0,0,Kay Mine 3_1.14.BK-77.xlsxKay Mine 3_1.14.BK-7...,0,0,0.0,12.0,Hole ConditioningRepairDrilling,Hole Conditioning had to flush the hole after...
1,01/14/2022,KM-21-59,Night,0,0,0,0,0,Kay Mine 3_1.14.BK-77.xlsxKay Mine 3_1.14.BK-7...,0,0,0.0,12.0,Hole ConditioningDrilling,Hole Conditioning Core HQ Diamond Core Bit fr...
2,01/14/2022,KM-22-57B,Day,0,0,0,0,4500 gal,Kay Mine 2_1.14.BK-67.xlsxKay Mine 2_1.14.BK-6...,0,0,0.0,12.0,DrillingSurveyHole Conditioning,WaterCore HQ Diamond Core Bit from 2501 ft to ...
3,01/14/2022,KM-22-57B,Night,0,0,0,0,8400 gal,Kay Mine 2_1.14.BK-67.xlsxKay Mine 2_1.14.BK-6...,0,0,0.0,12.0,Hole ConditioningDrillingSurvey,"Hole Conditioning drilled lost return, Core H..."
4,01/14/2022,KM-22-60,Day,0,0,0,0,0,Kay Mine 1_1.14.BK-75.xlsxKay Mine 1_1.14.BK-7...,0,0,0.0,12.0,Hole ConditioningSurveyDrilling,Hole Conditioning Surveys | Survey / Logging ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179,12/04/2022,KM-22-96,Aft,0,0,0,0,0,BK-77 12-4-22.xlsx,0,0,0.0,0.0,0,0
2180,12/04/2022,KM-22-96,B,0,0,0,0,0,BK-77 12-4-22.xlsx,0,0,0.0,0.0,0,0
2181,12/04/2022,KM-22-96,C,0,0,0,0,0,BK-77 12-4-22.xlsx,0,0,0.0,0.0,0,0
2182,12/04/2022,KM-22-96,Day,Hq,702,721,19,0,BK-77 12-4-22.xlsx,0,0,0.0,0.0,0,0


In [32]:
data_dict['Shift Data']=shifts
used_cols= list(set(common_cols+list(data_dict['Shift Data'].columns)))

In [33]:
non_shift= [c for c in all_df.columns if c not in used_cols]

non=all_df[common_cols+non_shift]
non=non.groupby(['Date','Hole ID']).sum(numeric_only=False).reset_index(level=[0,1])

In [34]:
data_dict['Non Shift Data']=non

In [35]:

write_excel(data_dict=data_dict,file=file)

In [36]:
safagf

NameError: name 'safagf' is not defined

In [ ]:
''

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.hist(X,bins=50)


In [ ]:
plt.hist(Y,bins=50)

In [ ]:
wb.activate()
wb.sheets.active.name='All Data'

wb.sheets.add("Info",after=True)
wb.sheets.add("Drilling",after=True)
wb.sheets.add("Hourly Distribution",after=True)
wb.sheets.add("Water",after=True)
wb.sheets.add("Products",after=True)
wb.sheets.add("Equipment",after=True)
wb.sheets.add("Bits",after=True)

In [ ]:
# scraping for the early version

In [ ]:
sheet=wb.sheets['All Data']
sheet["A4:N4"].value


In [ ]:
name_list=[]
data_list=[]

for i,file in enumerate(files):
    print(i,file)
    file_path=level2+file
    try:
        data=pd.read_excel(files,na_filter=True)
        #data['FileName']=file
        data_list.append(data)
        name_list.append(file_path)
    except Exception as e:
        print(e)

def get_nan_inds(series):
    ''' Obtain the first and last index of each consecutive NaN group.
    '''
    series = series.reset_index(drop=True)
    index = series[series.notna()].index.to_numpy()
    if len(index) == 0:
        return []
    indices = np.split(index, np.where(np.diff(index) > 1)[0] + 1)
     
    groups=[(ind[0], ind[-1]+1) for ind in indices]
    return groups

In [ ]:
def get_drill_data(data,indicators):
    data=data.T.reset_index(drop=True).T
    
    data_columns=[]
    ind_head_groups=[]
    for col in data.columns:
        blocks=get_nan_inds(data[col])
        #print(blocks)
        block_list=[]
        for rows in blocks:
            start=rows[0]
            end=rows[-1]
            if start==end:
                end+=1
            data_col=data.loc[list(range(start,end)),col]
            block_list.append(data_col)
            for thing in indicators:
                if thing in data_col.to_list():
                    size=rows[1]-rows[0]
                    print((thing,rows,size))
                    ind_head_groups.append((thing,size))

        try:
            non_nan=pd.concat(block_list,axis=0)
            data_columns.append(non_nan)
        except Exception as e:
            print(e)
    data=pd.concat(data_columns,axis=1,ignore_index=True).reset_index(drop=True)
    data=data.astype('object')

    headers=[]
    for i,thing in enumerate(indicators):
        header=data.where(data.eq(thing)).stack().index.values[0][0]
        if thing in ind_head_groups[i]:
            size=ind_head_groups[i][1]
            print(thing,'header:',header,'size:',size)
        headers.append((header,size))
    headers
    slices=[]
    for i,header in enumerate(headers):
        head=header[0]
        size=header[1]
        columns = data.iloc[head].str.lower()
        
        try:
            start=head+1
            end=start+size
            
            print(head,'-',end)
            slice = pd.DataFrame(data.values[start:end], columns=columns).reset_index(drop=True)
        except Exception as e: print('Exception:',e)
        slices.append(slice)
    clean_data=pd.concat(slices,axis=1)

    clean_data=clean_data.dropna(how='all',axis=1).T.dropna(how='all',axis=0).T

    clean_data=clean_data.T.groupby(level=0).first().T
    for col in clean_data.columns:
        try:
            clean_data[col]=clean_data[col].str.replace(' ','')
            clean_data[col]=clean_data[col].str.replace('**','',regex=False)
            clean_data[col]=clean_data[col].str.replace('!','',regex=False)
            clean_data[col]=clean_data[col].str.strip('ft')
            clean_data[col]=clean_data[col].str.strip('gal')
        except Exception as e: print(e)
        try:clean_data[col]=pd.to_numeric(clean_data[col])
        except Exception as e: print(e)
    clean_data['drill from']=pd.to_numeric(clean_data['drill from'],errors='coerce')
    drop=clean_data[clean_data['drill from'].isna()==True].index
    clean_data=clean_data.drop(drop)
    drop_col=['name','shift','size','tools']
    for col in drop_col:
        try:clean_data=clean_data.drop(col,axis=1).fillna(0)
        except Exception as e: print(e)
    clean_data['total man hours']=pd.to_numeric(clean_data['total man hours'],errors='coerce').fillna(method='ffill').fillna(method='bfill')
    date=data.where(data.eq('Date')).stack().index.values[0]
    clean_data['date']=data.iloc[date[0],date[1]+1]
    dr=data.where(data.eq('Drill')).stack().index.values[0]
    clean_data['drill']=data.iloc[dr[0],dr[1]+1]
    try:
        clean_data.rename(columns={'volume':'water_gal'},inplace=True)
    except Exception as e: print(e)
    return clean_data

In [ ]:
indicators=['Total Man Hours','Hole No.']
data=data_list[9]
data=get_drill_data(data,indicators=indicators)
data

In [ ]:
indicators=['Total Man Hours','Hole No.']
finaldata=[]
for data in data_list:
    data=get_drill_data(data=data,indicators=indicators)
    finaldata.append(data)
final=pd.concat(finaldata,axis=0)
final=final.sort_values(['hole no.','date','drill from']).reset_index(drop=True)
final_cols=['hole no.','date','drill from','drill to','water_gal']
others=[col for col in final.columns if col not in final_cols]
final=pd.concat([final[final_cols],final[others]],axis=1)
final.to_excel(outpath+'water_compilation.xlsx',index=False)
#final.to_excel(path+'water_comp.xlsx',index=False)
final